<a href="https://colab.research.google.com/github/citruscaesar/building-footprint-extraction/blob/main/experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Download Dependencies

In [ ]:
!pip install rasterio
!pip install lightning
!pip install torchmetrics
!pip install wandb
!pip install segmentation-models-pytorch
!clear

In [ ]:
!git clone https://github.com/citruscaesar/building-footprint-extraction.git
!mkdir -p /content/src
!cp -r /content/building-footprint-extraction/src/ /content/

Cloning into 'building-footprint-extraction'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 19 (delta 1), reused 15 (delta 0), pack-reused 0
Unpacking objects: 100% (19/19), 1.76 MiB | 1.14 MiB/s, done.


In [ ]:
!ls

building-footprint-extraction  sample_data  src


### Imports and Config

In [ ]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import rasterio as rio

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

import pytorch_lightning as pl
import torchmetrics 
from torchmetrics import MetricCollection
from torchmetrics.classification import BinaryJaccardIndex, BinaryConfusionMatrix, BinaryF1Score, BinaryCohenKappa

import wandb
from pytorch_lightning.loggers import WandbLogger, CSVLogger

import segmentation_models_pytorch as smp
from segmentation_models_pytorch.encoders import get_preprocessing_fn

from src.datamodule import InariaDataModule

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Pytorch Version: {torch.__version__}")
print(f"Lightning Version: {pl.__version__}")
print(f"Pytorch Running on :{DEVICE}")
print(torch.__config__.show())

Pytorch Version: 2.0.0+cu118
Lightning Version: 2.0.1
Pytorch Running on :cuda
PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.8
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75;-gencode;arch=compute_80,code=sm_80;-gencode;arch=compute_86,code=sm_86;-gencode;arch=compute_90,code=sm_90
  - CuDNN 8.7
  - Magma 2.6.1
  - Build settings: BLAS_INFO=mkl, BUILD_TYPE=Release, CUDA_VERSION=11.8, CUDNN_VERSION=8.7.0, CXX_COMPILER=/opt/rh/devtoolset-9/root/usr/bin/c++, CXX_FLAGS=

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/datasets/inria/archive/NEW2-AerialImageDataset.zip -d /content/drive/MyDrive/datasets/inria/

In [ ]:
DATA = Path("/content/drive/MyDrive/datasets/inria/train")
#DATA = Path("/media/sambhav/30AC4696AC46568E/datasets/inaria/train")

### Segmentation Module

In [ ]:
class SegModule(pl.LightningModule):
    def __init__(self, model, preprocess, learning_rate):
        super().__init__()
        self.model = model
        self.preprocess_fn = preprocess
        self.loss = smp.losses.DiceLoss(smp.losses.BINARY_MODE, from_logits = True)

        self.save_hyperparameters(ignore = ["model", "preprocess"])

        self.val_metrics = MetricCollection([
            BinaryF1Score(),
            BinaryJaccardIndex(),
            BinaryCohenKappa(),
        ], prefix = "Validation ")

        self.test_metrics = MetricCollection([
            BinaryF1Score(),
            BinaryJaccardIndex(),
            BinaryCohenKappa(),
        ], prefix = "Test ")


    def forward(self, batch):
        mask = self.model(batch) 
        return mask
        
    def training_step(self, batch, batch_idx):
        scene, mask = batch
        pred_mask = self.model(scene)
        loss = self.loss(pred_mask, mask) 
        self.log("train/loss", loss.item(), on_epoch = True, on_step=True, prog_bar=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        scene, mask = batch
        pred_mask = self.model(scene)
        loss = self.loss(pred_mask, mask)
        self.log("val/loss", loss.item(), on_epoch=True)

        self.val_metrics.update(pred_mask, mask)
        self.log_dict(self.val_metrics, on_epoch=True)

    def test_step(self, batch, batch_idx):
        scene, mask = batch
        pred_mask = self.model(scene)
        loss = self.loss(pred_mask, mask)
        self.log("test/loss", loss.item(), on_epoch=True)

        self.test_metrics.update(pred_mask, mask)
        self.log_dict(self.test_metrics, on_epoch=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.model.parameters(), lr = self.hparams.learning_rate)
        return optimizer

### Experiments

In [ ]:
pl.seed_everything(seed = 69, workers = True)

Global seed set to 69


69

In [ ]:
unet = smp.Unet("resnet18", encoder_weights="imagenet", in_channels=3, classes = 1)
preprocess = get_preprocessing_fn("resnet18", "imagenet")

In [ ]:
dm = InariaDataModule(DATA, batch_size = 8)

In [ ]:
os.environ["WANDB_NOTEBOOK_NAME"] = "experiments"
wandb.finish()
wandblogger = WandbLogger(
    project = "bulding-segmentation-tests",
    save_dir = "./logs/"
) 

In [ ]:
dev_trainer = pl.Trainer(
    accelerator = "gpu",
    fast_dev_run = 1
)

trainer = pl.Trainer(
    accelerator = "gpu",
    accumulate_grad_batches = 2,
    limit_train_batches = .25,
    max_epochs = 4,
    logger = [wandblogger],
    check_val_every_n_epochs = 1
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
model = SegModule(unet, preprocess, 3e-4)

In [ ]:
trainer.fit(model, datamodule = dm)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type             | Params
--------------------------------------------------
0 | model        | Unet             | 14.3 M
1 | loss         | DiceLoss         | 0     
2 | val_metrics  | MetricCollection | 0     
3 | test_metrics | MetricCollection | 0     
--------------------------------------------------
14.3 M    Trainable params
0         Non-trainable params
14.3 M    Total params
57.313    Total estimated model params size (MB)
/home/sambhav/bin/anaconda3/envs/ml/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1609: PossibleUserWarning: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_steps=1` reached.


In [ ]:
trainer.test(model, datamodule = dm)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         Testdice          │    0.8247647285461426     │
│          Testiou          │     0.701786994934082     │
│         Testkappa         │    0.7972214818000793     │
│         test/loss         │    0.22622278332710266    │
└───────────────────────────┴───────────────────────────┘

[{'test/loss': 0.22622278332710266,
  'Testdice': 0.8247647285461426,
  'Testiou': 0.701786994934082,
  'Testkappa': 0.7972214818000793}]

In [ ]:
wandb.finish()